# Email Reply Assistant with LM Studio
This notebook demonstrates how to connect to a local Language Model (LM) Studio server and use it to generate email replies.

In [ ]:
# Step 1: Install required libraries (if not already installed)
%pip install requests

In [ ]:
# Step 2: Import necessary libraries
import requests
import json

In [ ]:
# Step 3: Define connection details to your local LM Studio server
LM_STUDIO_URL = "http://localhost:1234/v1/chat/completions"

In [ ]:
# Step 4: Function to generate content using LM Studio
def generate_content(prompt, max_tokens=100, temperature=0.7, model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"):
    headers = {
        'Content-Type': 'application/json',
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return ""

In [ ]:
# Step 5: Load email content from a text file
FILE_NAME = "../email.txt"
with open(FILE_NAME, 'r') as file:
    incoming_email = file.read()
print("Incoming Email:")
print(incoming_email)

In [ ]:
# Step 6: Analyze the incoming email
analysis_prompt = f"Analyze the following email and extract the key points: {incoming_email}"
analysis = generate_content(analysis_prompt)
print("Analysis:")
print(analysis)

In [ ]:
# Step 7: Generate a response based on the analysis
response_prompt = f"Generate a polite and professional reply to the following email: {incoming_email}\nAnalysis: {analysis}"
response = generate_content(response_prompt, max_tokens=200)
print("Generated Response:")
print(response)

In [ ]:
# Step 8: Refine and proofread the generated response
refine_prompt = f"Refine and proofread the following email reply: {response}"
refined_response = generate_content(refine_prompt, max_tokens=150)
print("Refined Response:")
print(refined_response)